**Import external libraries.**

In [1]:
import os
import time
import torch
import torchvision.transforms as transforms
import numpy as np
import glob
import sys
from PIL import Image

**Mount the google drive.**

In [2]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


**Import network_architecture.py.**

In [3]:
filepath = "/content/drive/My Drive/CS376 Project"  #Change this path for your Google drive setting.
sys.path.append(filepath)

#from network_architecture import component_classifier
from network_architecture import component_classifier_function

**Unzip file containing the test dataset/images.**

In [4]:
!cp -r "{filepath}/test.zip" .
!unzip -q test.zip

**Util class for loading the test dataset.**

In [5]:
class component_test_dataset(torch.utils.data.Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.files = glob.glob(f"{root}/*.jpg")
    
    def __len__(self):
        ans = len(self.files)
        return ans

    def __getitem__(self, idx):
        image = Image.open(f"{self.root}/{str(idx + 1)}.jpg")
        if self.transform is not None:
            image = self.transform(image)
        return image

**Check the network design to make sure that the model was properly imported.**

In [6]:
experiment = "mobilenetv2_1500_0.001"  #Experiment name (for saving the result)
state_path = f"{filepath}/{experiment}/model_state_dict.pt"  #Model location

#model = component_classifier().cuda()
model = component_classifier_function().cuda()
print(model)
#model.load_state_dict(torch.load(state_path, map_location=torch.device('cpu')))
model.load_state_dict(torch.load(state_path))
model.eval()

# <All keys matched successfully>

MobileNetV2(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05,

MobileNetV2(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05,

**Load the test data.**

In [7]:
test_dataset = component_test_dataset(root=f"test",
                                                transform=transforms.Compose([
                                                    transforms.ToTensor()
                                                ]))
test_loader = torch.utils.data.DataLoader(test_dataset,
                                        batch_size=4,
                                        shuffle=False,
                                        drop_last=False
                                        )

**Obtain test accuracy using the trained network.**

In [8]:
prediction_index = []
accuracy_list = []

for _, (batch) in enumerate(test_loader):
    batch = batch.cuda()
    prediction = model(batch)
    _, i = prediction.max(dim=1)
    prediction_index += i.tolist()

final_index = prediction_index
print(f"Example prediction, 5 items from front : {final_index[:5]}")
print(f"""{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())} || Test finished!""")
#np.savetxt(f"{filepath}/{experiment}/test_result.csv", np.dstack((np.arange(1, len(final_index)+1), final_index))[0], "%d,%d", header="Id,Values")

file_name = "mobilenetv2_1500_0.001_1448"
np.savetxt(f"{filepath}/{experiment}/{file_name}.csv", np.dstack((np.arange(1, len(final_index)+1), final_index))[0], "%d,%d", header="Id,Values")

Example prediction, 5 items from front : [2, 2, 2, 2, 2]
2022-06-09 17:51:16 || Test finished!
